# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

from scipy.stats import linregress
# Import API key
from api_keys import g_key

import ipywidgets as widgets


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = os.path.join('..','WeatherPy', 'cities.csv')

In [3]:
weather_data = pd.read_csv(csvpath)
del weather_data['Unnamed: 0']
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Mitchell,1,US,1582157130,55,43.71,-98.03,10.40,11.41
1,Jamestown,75,US,1582156975,73,42.10,-79.24,26.01,9.17
2,Hamilton,1,US,1582157224,55,39.18,-84.53,39.20,3.36
3,Inuvik,90,CA,1582157225,85,68.35,-133.72,17.60,10.29
4,Iqaluit,20,CA,1582157008,59,63.75,-68.51,-31.00,3.36
...,...,...,...,...,...,...,...,...,...
558,Bilibino,100,RU,1582157419,94,68.05,166.44,-22.92,2.68
559,Vostok,100,RU,1582157419,92,46.49,135.88,7.20,9.10
560,Wenchi,61,GH,1582157419,62,7.74,-2.10,78.30,5.26
561,Pozo Colorado,93,PY,1582157420,72,-23.49,-58.80,82.60,2.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [5]:
# Create a list containing coordinates
coordinates = weather_data[["Lat", "Lon"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [6]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [7]:
#Add the layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)
fig.add_layer(heat_layer)

In [8]:
#Show figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# delete all rows with undesired temperatures
indexNames = weather_data[ (weather_data['Max Temp'] > 80)].index
weather_data.drop(indexNames , inplace=True)
indexNames = weather_data[ (weather_data['Max Temp'] < 70)].index
weather_data.drop(indexNames , inplace=True)

In [10]:
# delete all rows with undesired temperatures
indexNames = weather_data[ (weather_data['Cloudiness'] != 0) ].index
weather_data.drop(indexNames , inplace=True)
indexNames = weather_data[ (weather_data['Wind Speed'] >= 10) ].index
weather_data.drop(indexNames , inplace=True)

In [11]:
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
18,Richards Bay,0,ZA,1582157230,88,-28.78,32.04,76.77,6.15
84,Vung Tau,0,VN,1582157272,76,10.35,107.08,71.64,5.12
90,Narrabri,0,AU,1582157274,56,-30.32,149.78,75.00,5.82
144,Poya,0,NC,1582157290,74,-21.35,165.15,78.93,6.49
232,Januaria,0,BR,1582157198,84,-15.48,-44.37,71.38,0.83
317,Matara,0,LK,1582157347,86,5.95,80.54,70.56,5.59
325,Wagga Wagga,0,AU,1582157349,45,-35.12,147.37,70.00,4.70
362,Parli,0,IN,1582157286,48,18.85,76.53,72.19,7.65
401,Parana,0,AR,1582157349,43,-31.73,-60.52,79.00,1.99
554,Neuquen,0,AR,1582157418,13,-38.95,-68.06,78.80,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = weather_data

In [13]:
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"] = ""
hotel_df["Hotel Lng"] = ""

In [14]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
18,Richards Bay,0,ZA,1582157230,88,-28.78,32.04,76.77,6.15,,,
84,Vung Tau,0,VN,1582157272,76,10.35,107.08,71.64,5.12,,,
90,Narrabri,0,AU,1582157274,56,-30.32,149.78,75.00,5.82,,,
144,Poya,0,NC,1582157290,74,-21.35,165.15,78.93,6.49,,,
232,Januaria,0,BR,1582157198,84,-15.48,-44.37,71.38,0.83,,,
317,Matara,0,LK,1582157347,86,5.95,80.54,70.56,5.59,,,
325,Wagga Wagga,0,AU,1582157349,45,-35.12,147.37,70.00,4.70,,,
362,Parli,0,IN,1582157286,48,18.85,76.53,72.19,7.65,,,
401,Parana,0,AR,1582157349,43,-31.73,-60.52,79.00,1.99,,,
554,Neuquen,0,AR,1582157418,13,-38.95,-68.06,78.80,8.05,,,


In [15]:
from pprint import pprint

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "lodging",
    "radius" : 5000,
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # add keyword to params dict
    params['location'] = f"{row['Lat']}, {row['Lon']}"

     # assemble url and make API request
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #results[0]["name"]
    
    try:
        pprint(results[0]["name"])
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
        hotel_df.loc[index, 'Hotel Lat'] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, 'Hotel Lng'] = results[0]["geometry"]["location"]["lng"]
        

    except (KeyError, IndexError):
        print("--No hotel found for this city")



'BON Hotel Waterfront Richards Bay'
'Seaside Resort'
'Crossroads Hotel'
--No hotel found for this city
'Hotel Paraguassu'
'Amaloh Boutique Resort'
'The Club Motel and Apartments'
'Hotel Arya Executive'
'Howard Johnson Plaza Hotel Mayorazgo'
'Hotel del Comahue'


In [16]:
indexNames = hotel_df[ (hotel_df['Hotel Name'] == "") ].index
hotel_df.drop(indexNames , inplace=True)

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
18,Richards Bay,0,ZA,1582157230,88,-28.78,32.04,76.77,6.15,BON Hotel Waterfront Richards Bay,-28.7933,32.0808
84,Vung Tau,0,VN,1582157272,76,10.35,107.08,71.64,5.12,Seaside Resort,10.353,107.064
90,Narrabri,0,AU,1582157274,56,-30.32,149.78,75.00,5.82,Crossroads Hotel,-30.3231,149.782
232,Januaria,0,BR,1582157198,84,-15.48,-44.37,71.38,0.83,Hotel Paraguassu,-15.4815,-44.3667
317,Matara,0,LK,1582157347,86,5.95,80.54,70.56,5.59,Amaloh Boutique Resort,5.93921,80.5296
325,Wagga Wagga,0,AU,1582157349,45,-35.12,147.37,70.00,4.70,The Club Motel and Apartments,-35.1161,147.371
362,Parli,0,IN,1582157286,48,18.85,76.53,72.19,7.65,Hotel Arya Executive,18.8515,76.5262
401,Parana,0,AR,1582157349,43,-31.73,-60.52,79.00,1.99,Howard Johnson Plaza Hotel Mayorazgo,-31.7201,-60.5295
554,Neuquen,0,AR,1582157418,13,-38.95,-68.06,78.80,8.05,Hotel del Comahue,-38.9511,-68.0588


In [17]:
hotel_info = hotel_df[["Hotel Name", "City", "Country"]].values.tolist()

for i, s in enumerate(hotel_info):
    hotel_info[i] = f"""
    <b>Name</b>:<br>{s[0]}<br>
    <b>City</b>:<br>{s[1]}<br>
    <b>Country</b>:<br>{s[2]}
    """

In [18]:
locations = hotel_df[["Hotel Lat", "Hotel Lng"]].astype(float)

In [19]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable

markers = gmaps.marker_layer(
        locations,
        info_box_content=hotel_info
    )

# Add the layer to the map
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…